In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import sys 
import os 
import pyspark.sql.functions as f 
sys.path.append(os.path.abspath('../..'))
from data_loaders.data_load_excel import load_raw_data
from utils.file_utils import move_file
from data_writers.write_data import write_bronze_data

In [0]:
BASE_LANDING_PATH = "/Volumes/pei/default/landing_volume"
FILE_NAME = "Customer.xlsx"
TARGET_TABLE = "pei.bronze.raw_customers"

In [0]:
data_path = f"{BASE_LANDING_PATH}/customers/"
processed_path = f"{BASE_LANDING_PATH}/customers/processed/"
full_path = os.path.join(data_path, FILE_NAME)

# check if path exists
if os.path.exists(full_path):
    try: 
        # load data
        df_customer = load_raw_data(spark_session=spark, file_path=full_path)

        # write data
        write_bronze_data(
            df=df_customer, 
            target_table=TARGET_TABLE, 
            is_streaming=False, 
            file_source_path=full_path, 
            trigger=None
            )
       
       # move file to processed directory
       # TODO: this can lead to dual write problem, if move_file fails and write is succeeded
       #  We need to handle this programmatically or we can use autoloader with excel  
        move_file(spark, full_path, os.path.join(processed_path, FILE_NAME))
    except Exception as e: 
        print(f"FAILED: Customers Ingestion (Excel). Error: {str(e)}")
else:
    print(f"SKIPPED: {FILE_NAME} not found in {data_path}")